In [16]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [17]:
DICC_MES = {'Enero': '1', 'Febrero': '2', 'Marzo': '3', 'Abril': '4', 'Mayo': '5', 'Junio': '6', 'Julio': '7', 'Agosto': '8', 
            'Septiembre': '9', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'}
            
DICT_EQUIPOS = {}

In [18]:
LINK_GRUP_CHMPS = ["torneo=1068", "torneo=1067", "torneo=1066", "torneo=1065", "torneo=1073", "torneo=1086", 
                    "torneo=1104", "championsleague"]

In [19]:
def revisar_equipo(l, equipo):
    for i in l:
        eq = i.replace(",", "")
        eq = eq.replace(".", "")
        eq = eq.replace(" ", "")
        if eq[:3].upper() == equipo.upper():
            return i

In [25]:
def im_a_equipo(l):
    paises = []
    for i in l:
        im = i.find("img")
        im = im.get("src")
        im = im.split("/")[2].split(".")
        if im[1] == "png":
            im = "test"
        else:
            im = im[0]
            
        paises.append(im)
    return paises

In [21]:
locales = []
visitas = []
gol_locales = []
gol_visitas = []
ganador = []
goles = []
resultados = []
dif_goles = []
dias = []
anos = []
fases = []

In [22]:
ano = 2015
for num in LINK_GRUP_CHMPS:
    url = f"https://www.promiedos.com.ar/{num}"
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_grupos = sopa.find_all('div', class_ = 'grupo')
    for data in lista_grupos:
        equipos = data.find('table', id='posiciones')
        equipos = equipos.find_all('td', align = 'left')
        imagen = im_a_equipo(equipos)
        equipos = [i.text.replace(" ", "") for i in equipos]
        partidos = data.find_all('div', id = 'fixgrupo')

        for i in range(4):
            DICT_EQUIPOS[equipos[i]] = imagen[i]

        for datos in partidos:
            horario = datos.find_all('div', class_ = 'diahoragr')
            res = datos.find_all('div', class_ = 'grtr')

            for j in range(2):
                eq1 = res[j].find('div', class_ = 'greq1')
                eq1 = revisar_equipo(equipos, eq1.text)
                locales.append(eq1)
                eq2 = res[j].find('div', class_ = 'greq2')
                eq2 = revisar_equipo(equipos, eq2.text)
                visitas.append(eq2)

                resultado = res[j].find('div', class_ = 'grres4')
                resultado = resultado.text
                resultado = resultado.split('-')
                res1 = int(resultado[0])
                res2 = int(resultado[1])
                gol_locales.append(res1)
                gol_visitas.append(res2)
                resultados.append(str(res1) + "-" + str(res2))

                if res1 == res2:
                    ganador.append("Empate")
                    dif_goles.append(0)

                elif int(res1) > int(res2):
                    ganador.append("Local")
                    dif_goles.append(int(res1) - int(res2))

                elif int(res1) < int(res2):
                    ganador.append("Visita")
                    dif_goles.append(int(res2) - int(res1))

                goles.append(int(res1) + int(res2))
                anos.append(ano)
                dia = horario[j].text
                dia = dia.replace(",", "") ; dia = dia.split(" ")
                dia = dia[3]
                dias.append(dia)
                fases.append("Grupos")

    ano += 1


In [23]:
df_champions = pd.DataFrame()
df_champions["Local"] = locales
df_champions["Visita"] = visitas
df_champions["Gol_local"] = gol_locales
df_champions["Gol_visita"] = gol_visitas
df_champions["Ganador"] = ganador
df_champions["Resultado"] = resultados
df_champions["Año"] = anos
df_champions["Fase"] = fases
df_champions["Goles"] = goles
df_champions["Diferencia_goles"] = dif_goles
df_champions["Dia"] = dias
df_champions

,Local,Visita,Gol_local,Gol_visita,Ganador,Resultado,Año,Fase,Goles,Diferencia_goles,Dia
0,PSG,Malmo,2,0,Local,2-0,2015,Grupos,2,2,15/9
1,RealMadrid,Shakhtar,4,0,Local,4-0,2015,Grupos,4,4,15/9
2,Malmo,RealMadrid,0,2,Visita,0-2,2015,Grupos,2,2,30/9
3,Shakhtar,PSG,0,3,Visita,0-3,2015,Grupos,3,3,30/9
4,Malmo,Shakhtar,1,0,Local,1-0,2015,Grupos,1,1,21/10
...,...,...,...,...,...,...,...,...,...,...,...
763,PSG,Benfica,1,1,Empate,1-1,2022,Grupos,2,0,11/10
764,Benfica,Juventus,4,3,Local,4-3,2022,Grupos,7,1,25/10
765,PSG,MaccabiHaifa,7,2,Local,7-2,2022,Grupos,9,5,25/10
766,Juventus,PSG,1,2,Visita,1-2,2022,Grupos,3,1,2/11


In [26]:
for i in locales:
    if DICT_EQUIPOS[i] == 'test':
        print(i)